<a href="https://colab.research.google.com/github/franklinwanderson/scraping/blob/main/taxas_referenciais_scraping_b3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [1]:
!pip install -q kora

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 


In [2]:
import pandas as pd
from time import sleep
from kora.selenium import wd
import ssl
from datetime import date, datetime, timedelta
ssl._create_default_https_context = ssl._create_unverified_context

# Baixando Dados

> ### Função para Scraping

In [3]:
def BuscaB3(data = (date.today() - timedelta(days=1)).strftime("%Y-%m-%d"), TipoTaxa='PRE', espera=3):

  try: 
    url = f'http://www2.bmf.com.br/pages/portal/bmfbovespa/lumis/lum-taxas-referenciais-bmf-ptBR.asp?Data={date.fromisoformat(data).strftime("%d/%m/%Y")}&Data1={date.fromisoformat(data).strftime("%Y%m%d")}&slcTaxa={TipoTaxa}'
    wd.get(url)
    wd.find_element_by_link_text('Download').click()
    sleep(espera)
    arquivos= !ls -1t *.xls
    df = pd.read_html(arquivos[0].replace("'",''), encoding='ISO-8859-1', thousands='.', decimal=',' )[0]
    df.columns = ['Dias_Corridos', 'Curva_252', 'Curva_360']
    df.loc[:,'Data'] = pd.to_datetime(data)
    df.set_index('Data', inplace=True)
    df.dropna(inplace = True)
    
  except:
    df = pd.DataFrame()

  return df

 

> ### Rodando scraping para 1 dia

In [7]:
df = BuscaB3(data = '2021-06-16', TipoTaxa = 'PRE', espera = 1.5)
df 

,Dias_Corridos,Curva_252,Curva_360
Data,,,
2021-06-16,1.0,3.40,0.00
2021-06-16,2.0,3.79,5.46
2021-06-16,7.0,4.03,4.12
2021-06-16,13.0,4.10,4.06
2021-06-16,14.0,4.11,4.20
...,...,...,...
2021-06-16,10288.0,9.70,9.52
2021-06-16,10471.0,9.70,9.53
2021-06-16,10652.0,9.71,9.53



> ### Rodando scraping para um período

In [8]:
data_inicial = '6/4/2021' #mes / dia / ano
data_final = '6/16/2021' #mes / dia / ano

df = pd.DataFrame()

for d in pd.bdate_range(start = data_inicial, end = data_final):
  df = df.append(BuscaB3(data = d.strftime("%Y-%m-%d"), TipoTaxa = 'PRE', espera = 3))

df

,Dias_Corridos,Curva_252,Curva_360
Data,,,
2021-06-04,3.0,3.40,0.00
2021-06-04,7.0,3.74,3.82
2021-06-04,11.0,3.76,3.41
2021-06-04,12.0,3.77,3.59
2021-06-04,13.0,3.78,3.73
...,...,...,...
2021-06-16,10288.0,9.70,9.52
2021-06-16,10471.0,9.70,9.53
2021-06-16,10652.0,9.71,9.53


> ### Download

In [6]:
df.to_excel('taxas_ref_b3.xlsx')